In [1]:
%load_ext autoreload
%autoreload 2

In [48]:
from math import sin

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tqdm

In [155]:
# functions to test
def parity(x):
    return x % 2

def mysin(x):
    return sin(x)

# makes two-column dataset, first is data input to function of choice, second gets replaced w/ function output
def make_xs(n):
    return np.random.randint(0, 10, (n, 2))

# calls function of choice, f
def make_data(n, f, dtype=None):
    xs = make_xs(n) if dtype is None else make_xs(n).astype(dtype)
    xs[:, 1] = f(xs[:, 0])
    xs, ys = xs[:, 0], xs[:, 1]
    return xs, ys


# TODO: add batches
def make_data_parity(n):
    xs_train, ys_train = make_data(n, parity)
    xs_test, ys_test = make_data(n // 10, parity)
    return xs_train, ys_train, xs_test, ys_test


def make_data_sin(n):
    xs_train, ys_train = make_data(n, np.sin, dtype=np.float32)
    xs_test, ys_test = make_data(n // 10, np.sin, dtype=np.float32)
    return xs_train, ys_train, xs_test, ys_test


def make_loader(xs, ys, batch_size):
    xs, ys = torch.tensor(xs, dtype=torch.float32, requires_grad=True), torch.tensor(ys, dtype=torch.float32, requires_grad=True)
    data = list(zip(xs, ys))
    loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
    return loader


def euclidean_distance(x, y):
    return torch.sqrt(torch.sum((x - y) ** 2))


def accuracy(x, y):
    acc = torch.sum(x == y) / len(x)
    return acc.item()


n = 1000

xs_train_parity, ys_train_parity, xs_test_parity, ys_test_parity = make_data_parity(n)
xs_train_sin, ys_train_sin, xs_test_sin, ys_test_sin = make_data_sin(n)


print(list(zip(xs_train_parity[:5], ys_train_parity[:5])))

print(list(zip(xs_train_sin[:5], ys_train_sin[:5])))

[(np.int64(1), np.int64(1)), (np.int64(5), np.int64(1)), (np.int64(4), np.int64(0)), (np.int64(8), np.int64(0)), (np.int64(0), np.int64(0))]
[(np.float32(1.0), np.float32(0.841471)), (np.float32(9.0), np.float32(0.41211846)), (np.float32(8.0), np.float32(0.98935825)), (np.float32(5.0), np.float32(-0.9589243)), (np.float32(0.0), np.float32(0.0))]


In [164]:
class MLP(nn.Module):
    def __init__(self, act=F.relu):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(1, 10)
        self.fc2 = nn.Linear(10, 1)
        self.act = act
    
    def forward(self, x):
        if isinstance(x, np.ndarray):
            return self.forward(torch.tensor(x, dtype=torch.float32).unsqueeze(1))
        
        x = x.unsqueeze(1)
        
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        return x
    

def train_step(model, loss_fn, opt, x, y):
    assert model.training and x.requires_grad and y.requires_grad
    opt.zero_grad()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    loss.backward()
    opt.step()
    return y_pred, loss.item()
    

def train(model, loss_fn, opt, loader, epochs=10):
    for epoch in (pbar := tqdm.trange(epochs, desc="loss: inf, acc: inf")):
        for x, y in loader:
            y_pred, loss = train_step(model, loss_fn, opt, x, y)
            acc = accuracy(y_pred, y)
            pbar.set_description(f"loss: {loss:.3f}, acc: {acc:.3f}")
    
lr = 1e-3

model = MLP(); model.train()
opt = optim.SGD(model.parameters(), lr=lr)
loader_parity = make_loader(xs_train_parity, ys_train_parity, 32)

In [150]:
x, y = next(iter(loader_parity))

train_step(model, euclidean_distance, opt, x, y)

23.124238967895508

In [167]:
epochs = 100

train(model, euclidean_distance, opt, loader_parity, epochs=epochs)

loss: 4.003, acc: 0.000: 100%|██████████| 100/100 [00:09<00:00, 10.79it/s]
